<a href="https://colab.research.google.com/github/corrine1106/clv_pred/blob/main/clvEncDecSeqModelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadr

In [ ]:
from keras.engine.input_layer import Input
from tf.keras.layers import Lambda, Embedding, Reshape, Conv1D, Concatenate, GRU, Dense, TimeDistributed
import tensorflow as tf


In [ ]:
import pandas as pd
import numpy as np
import os.path
import pyreadr
import json 

ModuleNotFoundError: ignored

In [ ]:
allSeqFeatLayerLength = seqInLength

# 用list comprehension轉換match function in R
seqFeatIndVec = [ embeddingFeatVec.index(i) for i in seqFeatVec]

if (length(embeddingFeatVec) > 0) and (!useSkipGramEmbeddings):
    # ref:https://www.geeksforgeeks.org/create-a-pandas-dataframe-from-lists/
    # data.table會把不足的值重複，但是zip不會 不知道會不會影響
    embeddingConfigDT = pd.DataFrame(list(zip(embeddingFeatVec,seqFeatIndVec,
                                              apply(lambda ind: max(X_DT[[embeddingFeatVec[ind]]]) + 1,range(len(embeddingFeatVec))),
                                              embedDimVec,[allSeqFeatLayerLength]*len(embeddingFeatVec))),
               columns =['seq_feat', 'seq_feat_ind','input_dim','output_dim','input_length'])
else:
    # R 可以創一個空的df但是pandas好像不行，所以我先創一個跟上面一樣columns的df
    embeddingConfigDT = pd.Dataframe(columns =['seq_feat', 'seq_feat_ind','input_dim','output_dim','input_length'])

if useSkipGramEmbeddings:
    # paste0把兩個接起來
    # example:
    # nth <- paste0(1:12, c("st", "nd", "rd", rep("th", 9)))
    #  [1] "1st"  "2nd"  "3rd"  "4th"  "5th"  "6th"  "7th"  "8th"  "9th"  "10th" "11th"
    # [12] "12th"
    if os.path.isfile(path+skipGramEmbeddingFilename):
        # readRDS是讀取讀取rsd檔案（R物件）
        # ref:https://stackoverflow.com/questions/40996175/loading-a-rds-file-in-pandas
        skipGramEmbeddingMatrix = pyreadr.read_r(path+skipGramEmbeddingFilename)
    # shape because it's column
    embedDimVec = [skipGramEmbeddingMatrix.shape[1]]*len(embeddingFeatVec)
    seqFeatIndVec = [ embeddingFeatVec.index(i) for i in seqFeatVec]
    skipGramNumWords = skipGramEmbeddingMatrix.shape[0]
    embeddingConfigDT = pd.DataFrame(list(zip(embeddingFeatVec,seqFeatIndVec,
                                              skipGramNumWords,
                                              embedDimVec,[allSeqFeatLayerLength]*len(embeddingFeatVec))),
               columns =['seq_feat', 'seq_feat_ind','input_dim','output_dim','input_length'])
# 原match function: match(convFeatVec, seqFeatVec)
seqFeatIndVec = [ convFeatVec.index(i) for i in seqFeatVec]


if len(convFeatVec) > 0:
    convConfigDT = pd.DataFrame(list(zip(convFeatVec,seqFeatIndVec,
                                              filterVec,
                                              kernelSizeVec,[convPaddingType]*len(convFeatVec))),
               columns =['seq_feat', 'seq_feat_ind','filters','kernel_size','padding'])

else:
    convConfigDT = pd.Dataframe(columns =['seq_feat', 'seq_feat_ind','filters','kernel_size','padding'])


if len(convFeatVec) > 0:
    convConfigLayer2DT = pd.DataFrame(list(zip(convFeatVec,seqFeatIndVec,
                                              filterLayer2Vec,
                                              kernelSizeLayer2Vec,[convPaddingType]*len(convFeatVec))),
               columns =['seq_feat', 'seq_feat_ind','filters','kernel_size','padding'])

else:
    convConfig2DT = pd.Dataframe(columns =['seq_feat', 'seq_feat_ind','filters','kernel_size','padding'])

# layer_input應該是input layer的意思 指定input的規格
allSeqFeatLayerInputs = Input(shape = (allSeqFeatLayerLength, numSeqFeat), name = 'allSeqFeatLayerInputs')
# > rep("", 3)
# [1] "" "" ""
# 這個形式沒有直接對應的
# https://www.r-bloggers.com/2014/02/character-strings-in-r/
# 應該可以當作list of string 看待
sliceSeqFeatNameVec = [""]*numSeqFeat

sliceSeqFeatLayerList = [np.NaN]

for sliceInd in range(numSeqFeat):
    sliceSeqFeatNameVec[sliceInd] = "sliceSeqFeat_" + seqFeatVec[sliceInd]
    sliceSeqFeatLayerList[sliceInd] = Lambda(function = lambda x: x[, , sliceInd], name = "seqFeatVec[%s]"%(sliceInd)))(allSeqFeatLayerInputs)

sliceSeqFeatLayerInputList = sliceSeqFeatLayerList

sliceEmbedFeatNameVec = [""]*embeddingConfigDT.shape[0]
sliceEmbedFeatLayerList = [np.NaN]

if len(embeddingConfigDT) > 0:
    for embFeatInd in range(len(embeddingConfigDT)) :

        sliceEmbedFeatNameVec[embFeatInd] = "sliceEmbedFeat_" + embeddingConfigDT.loc[embFeatInd,"seq_feat"]
        sliceEmbedFeatNameVec[embFeatInd] = Embedding(  input_dim       =   embeddingConfigDT.loc[embFeatInd,'input_dim'],
                                                        output_dim      =   embeddingConfigDT.loc[embFeatInd,'output_dim'],
                                                        input_length    =   embeddingConfigDT.loc[embFeatInd,'input_length'],
                                                        name            =   embeddingConfigDT.loc[embFeatInd,'seq_feat']+'_embed')(sliceSeqFeatLayerList[embeddingConfigDT.loc[embFeatInd,"seq_feat_ind"]])
        sliceEmbedFeatLayerList[embeddingConfigDT.loc[embFeatInd,'seq_feat_ind']] = sliceEmbedFeatNameVec[embFeatInd]
        sliceSeqFeatLayerList[embeddingConfigDT.loc[embFeatInd,'seq_feat_ind']] = sliceEmbedFeatNameVec[embFeatInd]

# > setdiff(1:6, 2:4)
# [1] 1 5 6
# ref:https://stackoverflow.com/questions/3462143/get-difference-between-two-lists
for reshapeInd in list(set(range(len(seqFeatVec)))-set(embeddingConfigDT['seq_feat_ind'])):
    sliceSeqFeatLayerList[reshapeInd] =  Reshape(target_shape = (dim(sliceSeqFeatLayerList[reshapeInd])[2], 1), 
                                                                            name = seqFeatVec[reshapeInd]+'_reshape')(sliceSeqFeatLayerList[reshapeInd])

sliceConvFeatNameVec = [""]*len(convConfigDT)

if len(convConfigDT) > 0:
    for convFeatInd in range(len(convConfigDT)):
        sliceConvFeatNameVec[convFeatInd] = "sliceConvFeat_"+convConfigDT.loc[convFeatInd,'seq_feat']
        sliceConvFeatNameVec[convFeatInd] = Conv1D(  filters     =   convConfigDT.loc[convFeatInd,'filters'],
                                                     kernel_size =   convConfigDT.loc[convFeatInd,'kernel_size'],
                                                     padding     =   convConfigDT.loc[convFeatInd,'padding'],
                                                     name        =   convConfigDT.loc[convFeatInd,'seq_feat'] + '_conv'))(sliceSeqFeatLayerList[convConfigDT.loc[convFeatInd,seq_feat_ind]])
        
    if !useSeparateConv:
        sliceSeqFeatLayerList[convConfigDT.loc[convFeatInd,'seq_feat_ind']] = sliceConvFeatNameVec[convFeatInd]

sliceConvLayer2FeatNameVec = [""]*len(convConfigLayer2DT)

if len(convConfigLayer2DT) > 0:
    for convFeatInd in range(len(convConfigLayer2DT)):
        sliceConvLayer2FeatNameVec[convFeatInd] = "sliceConvLayerFeat_"+convConfigLayer2DT.loc[convFeatInd,'seq_feat']
        if (!useSeparateConv):
            sliceConvLayer2FeatNameVec[convFeatInd] = Conv1D( filters     =   convConfigLayer2DT.loc[convFeatInd,'filters'],
                                                              kernel_size =   convConfigLayer2DT.loc[convFeatInd,'kernel_size'],
                                                              padding     =   convConfigLayer2DT.loc[convFeatInd,'padding'],
                                                              name        =   convConfigLayer2DT.loc[convFeatInd,'seq_feat'] + '_conv_layer2'))(sliceSeqFeatLayerList[convConfigLayer2DT.loc[convFeatInd,seq_feat_ind]])
                
            sliceSeqFeatLayerList[convConfigLayer2DT.loc[convFeatInd,'seq_feat_ind']] = sliceConvLayer2FeatNameVec[convFeatInd]

        else:
            sliceConvLayer2FeatNameVec[convFeatInd] = Conv1D( filters     =   convConfigLayer2DT.loc[convFeatInd,'filters'],
                                                              kernel_size =   convConfigLayer2DT.loc[convFeatInd,'kernel_size'],
                                                              padding     =   convConfigLayer2DT.loc[convFeatInd,'padding'],
                                                              name        =   convConfigLayer2DT.loc[convFeatInd,'seq_feat'] + '_conv_layer2'))(sliceConvFeatNameVec[convFeatInd])


if len(seqFeatVec) > 1:
    if (!useSeparateConv):
        encoderSeqFeat = Concatenate(axis = 2)(sliceSeqFeatLayerList)
    else:
        sliceSeqFeatLayerAddConvList = sliceSeqFeatLayerList
        currLength = len(sliceSeqFeatLayerAddConvList)

    if len(convConfigDT) > 0:
        for convFeatInd in range(len(convConfigDT)):
            sliceSeqFeatLayerAddConvList[currLength + convFeatInd] = sliceConvFeatNameVec[convFeatInd]

    currLength = len(sliceSeqFeatLayerAddConvList)

    if len(convConfigLayer2DT) > 0:
        for convFeatInd in range(len(convConfigLayer2DT)):
            sliceSeqFeatLayerAddConvList[currLength + convFeatInd] = sliceConvLayer2FeatNameVec[convFeatInd]

    encoderSeqFeat = Concatenate(axis = 2)(sliceSeqFeatLayerAddConvList)

else:
    encoderSeqFeat = sliceSeqFeatLayerList[1]

encoderGRULayer = GRU(units = nUnitsEncoder, dropout = dropoutEncoder, recurrent_dropout = recurrentDropoutEncoder, 
                                        activation = activationGRU, kernel_initializer = kernelInitGRU, return_state = TRUE)

encoderOutputList =  encoderGRULayer(encoderSeqFeat)
hiddenStatesEncoderGRU = encoderOutputList[2]


if (nUnitsEncoder_2 > 0):
    nUnitsEncoder_2 = nUnitsEncoder
    hiddenStatesEncoderGRU_1 = hiddenStatesEncoderGRU
    if (useDenseEncoder1ToEncoder2Layer):
        hiddenStatesEncoderGRU_1 = Dense(units = nUnitsEncoder_2, activation = "tanh")(hiddenStatesEncoderGRU_1)

    encoder_2_outputList = GRU(units = nUnitsEncoder_2, dropout = dropoutEncoder, recurrent_dropout = recurrentDropoutEncoder, 
                                            activation = activationGRU, kernel_initializer = kernelInitGRU,
                                                return_state = TRUE)(initial_state = hiddenStatesEncoderGRU_1)(encoderSeqFeat)
    hiddenStatesEncoderGRU_2 = encoder_2_outputList[[2]]
    hiddenStatesEncoderGRU = hiddenStatesEncoderGRU_2

if (useDenseEncoderToDecoderLayer):
    hiddenStatesEncoderGRU = Dense(units = nUnitsDecoder, activation = "tanh")(hiddenStatesEncoderGRU)

decoderSeqFeat = encoderSeqFeat

decoderSliceOutput = Lambda(function = lambda xVal: xVal[, (seqInLength - seqOutLength + 1):seqInLength, ])

decoderOutputs = decoderSliceOutput(TimeDistributed(layer = Dense(units = 1))(GRU(units = nUnitsDecoder, dropout = dropoutDecoder, recurrent_dropout = recurrentDropoutDecoder, 
                                    activation = activationGRU, kernel_initializer = kernelInitGRU,
                                        return_sequences = TRUE,initial_state = hiddenStatesEncoderGRU)(decoderSeqFeat)))

encoderDecoderModel = tf.keras.Model(
    inputs      =   allSeqFeatLayerInputs,
    outputs     =   decoderOutputs)

if (useSkipGramEmbeddings):
    encoderDecoderModel.get_layer(name = "CustomerID_embed").set_weights(list(skipGramEmbeddingMatrix)).trainable = False

encoderDecoderModel.compile(optimizer = optimizerVal,
                            loss      = lossVal)

if (!useTrainGen & sampleFracCust == 1 & sampleFracItem == 1):
    # rm = delete objects from the memory
    del X_DT, customerFeaturesDT_all, subcategoryNumberOfOrdersSparseMatrix_all, customerOrderTimeDT
    # gc() 好像python會自動清除垃圾 ref:https://www.tutorialspoint.com/How-does-garbage-collection-work-in-Python

if (continueTrain):
    encoderDecoderModel.load_weights(path + "encoderDecoderModel_full_current_" + dataset + ".hdf5")

# rowSums meaning ref:https://www.geeksforgeeks.org/compute-the-sum-of-rows-of-a-matrix-or-array-in-r-programming-rowsums-function/
totalTrueVec = np.sum(Y_valid_Array[:,:, 1], axis=1)
finalRmseTotalVec = 0

if (use2Valid):
    totalTrueVec_2 = np.sum(Y_valid_Array_2[:,:, 1], axis=1)
    finalRmseTotalVec_2 = 0

if (useTrainGen):
    trainHistory = encoderDecoderModel.fit_generator(
            generator           =   trainGeneratorSeqCustom(X_DT, batch_size = batchSizeVal), 
            steps_per_epoch     =   1,
            epochs              =   nEpochVal,
            max_queue_size      =   1,
            validation_data     =   list(list(X_valid_Array), Y_valid_Array[, , 1, drop = FALSE]),
            verbose             =   0,
            callbacks           =   list(callback_early_stopping(monitor = "val_loss", patience = earlyStoppingPatienceVal),
                                         callback_model_checkpoint(paste0(path, "best_model_", tryOutIt, "_", runTimestamp, ".h5"), 
                                         save_best_only = TRUE, save_weights_only = TRUE)))

    validLossVec = trainHistory[metrics][val_loss]

else:
    if (repeatSampleFit):
        validLossVec = 0
        for epochVal in range(nEpochVal):
            if (epochVal > 0):
                if (sampleFracCust < 1 | sampleFracItem < 1):
                    # clvGenTrainArrays.r裡面的裡面的function
                    trainGenSourceFun(passToGlobEnv = TRUE)

            trainHistory = encoderDecoderModel.fit(
                    x               =   list(X_train_Array),
                    y               =   Y_train_Array[, , 1, drop = FALSE],
                    epochs          =   1,
                    batch_size      =   batchSizeVal,
                    validation_data =   list(list(X_valid_Array), Y_valid_Array[, , 1, drop = FALSE]),
                    verbose         =   0,
                    callbacks       =   list(callback_early_stopping(monitor = "val_loss", patience = earlyStoppingPatienceVal),
                                             callback_model_checkpoint(paste0(path, "best_model_", tryOutIt, "_", runTimestamp, ".h5"), 
                                             save_best_only = TRUE, save_weights_only = TRUE))
                )
            
            validLossVec = [validLossVec, trainHistory[metrics][val_loss]]
            
            y_valid_pred_Array = encoderDecoderModel.predict(X_valid_Array)
            totalPredVec = np.sum(y_valid_pred_Array, axis=1)
            finalRmseTotal = np.sqrt(mean((totalPredVec - totalTrueVec)^2))
            finalRmseTotalVec = [finalRmseTotalVec, finalRmseTotal]
            
            if (use2Valid):
                y_valid_pred_Array_2 = encoderDecoderModel.predict(X_valid_Array_2)
                totalPredVec_2 = np.sum(y_valid_pred_Array_2, axis=1)
                finalRmseTotal_2 = np.sqrt(mean((totalPredVec_2 - totalTrueVec_2)^2))
                finalRmseTotalVec_2 = [finalRmseTotalVec_2, finalRmseTotal_2]
            
            if ((lossVal == "mean_squared_error") & (epochVal == 1 | min(finalRmseTotalVec) == finalRmseTotal)):
                encoderDecoderModel.save_weights(filepath = path+"encoderDecoderModel_currentbest_"+dataset+"_"+tryOutIt+"_"+runTimestamp+".h5",
                                                 overwrite = TRUE)

            if ((np.argmin(validLossVec) < len(validLossVec) - earlyStoppingPatienceVal) | 
                (epochVal > stopIfErrorMoreThanPrevFactorDelay & 
                    validLossVec[len(validLossVec)] >= min(validLossVec[-len(validLossVec)]) * stopIfErrorMoreThanPrevFactor)):
                break
            
    else:
        trainHistory = encoderDecoderModel.fit(   x               = list(X_train_Array),
                                                  y               = Y_train_Array[, , 1, drop = FALSE],
                                                  epochs          = nEpochVal,
                                                  batch_size      = batchSizeVal,
                                                  validation_data = list(list(X_valid_Array), Y_valid_Array[, , 1, drop = FALSE]),
                                                  verbose         = 0,
                                                  callbacks       = list(callback_early_stopping(monitor = "val_loss", patience = earlyStoppingPatienceVal),
                                                                        callback_model_checkpoint(paste0(path, "best_model_", tryOutIt, "_", runTimestamp, ".h5"), 
                                                                        save_best_only = TRUE, save_weights_only = TRUE)))
            
        validLossVec = trainHistory[metrics][val_loss]
    

encoderDecoderModel.save_weights(filepath = path+"encoderDecoderModel_full_"+dataset+"_"+tryOutIt+"_"+runTimestamp+".h5",
                                                 overwrite = TRUE)

try:
    encoderDecoderModel.load_weights(path + "encoderDecoderModel_currentbest_"+dataset+"_"+tryOutIt+"_"+runTimestamp+".h5")

y_valid_pred_Array = encoderDecoderModel.predict(X_valid_Array)
preds_valid_DT = temp_subslice_X_valid_wide_DT.loc[:, 'iterateVarVec']

# ref:https://stackoverflow.com/questions/39050539/how-to-add-multiple-columns-to-pandas-dataframe-in-one-assignment
preds_valid_DT = pd.concat([preds_valid_DT,pd.DataFrame([[y_valid_pred_Array.reshape(y_valid_pred_Array.shape[0], y_valid_pred_Array.shape[1])]],columns=["pred_"+i for i in range(y_valid_pred_Array.shape[1]) ])],axis=1)
true_valid_DT = temp_subslice_X_valid_wide_DT.loc[:, 'iterateVarVec']
true_valid_DT = pd.concat([true_valid_DT,pd.DataFrame([[Y_valid_Array[:,:,1].reshape(Y_valid_Array[:,:,1].shape[0],Y_valid_Array[:,:,1].shape[1])]],columns=["true_"+i for i in range(Y_valid_Array.shape[1]) ])],axis=1)

if (use2Valid):
    y_valid_pred_Array_2 = encoderDecoderModel.predict(X_valid_Array_2)
    preds_valid_DT_2 = temp_subslice_X_valid_wide_DT_2.loc[:,'iterateVarVec']
    preds_valid_DT_2 = pd.concat([preds_valid_DT_2,pd.DataFrame([[y_valid_pred_Array_2.reshape(y_valid_pred_Array_2.shape[0], y_valid_pred_Array_2.shape[1])]],columns=["pred_"+i for i in range(y_valid_pred_Array_2.shape[1]) ])],axis=1)
    true_valid_DT = temp_subslice_X_valid_wide_DT_2.loc[:, 'iterateVarVec']
    true_valid_DT = pd.concat([true_valid_DT_2,pd.DataFrame([[Y_valid_Array_2[:,:,1].reshape(Y_valid_Array_2[:,:,1].shape[0],Y_valid_Array_2[:,:,1].shape[1])]],columns=["true_"+i for i in range(Y_valid_Array_2.shape[1]) ])],axis=1)

try:
    preds_valid_DT.to_csv(path+''.join("predsEncDec_valid_DT_", dataset, "_", tryOutIt, ".csv"))
    true_valid_DT.to_csv(''.join(path, "trueEncDec_valid_DT_", dataset, "_", tryOutIt, ".csv"))

if (use2Valid):
    try:
        preds_valid_DT_2.to_csv(path+''.join("predsEncDec_valid_DT_2_", dataset, "_", tryOutIt, ".csv"))
        true_valid_DT_2.to_csv(''.join(path, "trueEncDec_valid_DT_2_", dataset, "_", tryOutIt, ".csv"))



finalRmseWeekwise = np.sqrt(mean((y_valid_pred_Array - Y_valid_Array[, , 1])**2))

finalMseWeekwise = mean((y_valid_pred_Array - Y_valid_Array[, , 1])**2)

totalTrueVec = np.sum(Y_valid_Array[:,:, 1], axis=1)

totalPredVec = np.sum(y_valid_pred_Array, axis=1) 

finalRmseTotal = np.sqrt(mean((totalPredVec - totalTrueVec)**2))

if (use2Valid):
    min(finalRmseTotalVec_2)

finalMseTotal = np.mean((totalPredVec - totalTrueVec)**2)

finalRmseTotal_2 = finalRmseTotal

if (use2Valid):

    totalTrueVec_2 = np.sum(Y_valid_Array_2[:,:, 1], axis=1)
    totalPredVec_2 = np.sum(y_valid_pred_Array_2, axis=1)
    finalRmseTotal_2 = np.sqrt(mean((totalPredVec_2 - totalTrueVec_2)**2))
    finalMseTotal_2 = mean((totalPredVec_2 - totalTrueVec_2)**2)

minValidLossVal = min(validLossVec)
argminValidLossVal = np.argmin(validLossVec)

# 原本是list我改用我改用dict存
saveParamResultList = {
    dataset                             = dataset,
    runTimestamp                        = runTimestamp,
    tryOutIt                            = tryOutIt,
    continueTrain                       = continueTrain,
    sampleFracCust                      = sampleFracCust,
    sampleFracItem                      = sampleFracItem,
    timeVarName                         = timeVarName,
    minTime                             = minTime,
    maxTime                             = maxTime,
    minValidTime                        = minValidTime,
    maxValidTime                        = maxValidTime,
    minTestTime                         = minTestTime,
    maxTestTime                         = maxTestTime,
    seqInLength                         = seqInLength,
    seqOutLength                        = seqOutLength,
    usePeriodicTrainData                = usePeriodicTrainData,
    useSkipGramEmbeddings               = useSkipGramEmbeddings,
    minRmseTotal                        = min(finalRmseTotalVec),
    minRmseTotal_2                      = min(finalRmseTotalVec_2),
    finalRmseTotalVec                   = paste(as.character(finalRmseTotalVec), collapse = " "),
    finalRmseTotalVec_2                 = paste(as.character(finalRmseTotalVec_2), collapse = " "),
    convPaddingType                     = convPaddingType,
    useSeparateConv                     = useSeparateConv,
    nUnitsEncoder                       = nUnitsEncoder,
    nUnitsEncoder_2                     = nUnitsEncoder_2,
    nUnitsDecoder                       = nUnitsDecoder,
    dropoutEncoder                      = dropoutEncoder,
    recurrentDropoutEncoder             = recurrentDropoutEncoder,
    dropoutDecoder                      = dropoutDecoder,
    recurrentDropoutDecoder             = recurrentDropoutDecoder,
    learnRateVal                        = learnRateVal,
    nEpochVal                           = nEpochVal}

try:
    # 最後面感覺只是存資料資料 我先寫成存json檔
    with open("saveParamResult.json", "w") as outfile:
        json.dump(saveParamResultList, outfile)

SyntaxError: ignored